In [ ]:
!unzip /content/drive/MyDrive/food_classifier_data.zip

Archive:  /content/drive/MyDrive/food_classifier_data.zip
replace food_classifier_data/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
train = pd.read_csv('/content/food_classifier_data/Training_set_food.csv')
#test = pd.read_csv('dataset/test.csv')
print(train.head())

      filename    label
0  Image_1.jpg  burgers
1  Image_2.jpg  burgers
2  Image_3.jpg  burgers
3  Image_4.jpg  burgers
4  Image_5.jpg  burgers


In [ ]:

from fastai.imports import *
from fastai import *
from fastai.vision import *
from torchvision.models import *

In [ ]:
path='/content/food_classifier_data'
tfms = get_transforms(do_flip = True, max_lighting = 0.2, max_zoom= 1.1, max_warp = 0.15, max_rotate = 45)
data = ImageDataBunch.from_csv(path, folder= 'train', 
                              valid_pct = 0.0,
                              csv_labels = 'Training_set_food.csv',
                              ds_tfms = tfms, 
                              fn_col = 'filename',
                              test = '/content/food_classifier_data/test', 
                              label_col = 'label',
                              bs = 32,
                              size = 224).normalize(imagenet_stats)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpola

In [ ]:
import warnings
warnings.filterwarnings('ignore')
fbeta = FBeta(average='weighted', beta = 1)
learn = cnn_learner(data, models.densenet121, metrics=[accuracy, fbeta])
learn.fit_one_cycle(6, max_lr =[8e-6, 8e-4, 8e-3] )#slice(8e-6, 8e-3)
  #Finding learning rate based on the training

In [ ]:
test=pd.read_csv("/content/food_classifier_data/Testing_set_food.csv")

In [ ]:
test

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg
...,...
1795,Image_1796.jpg
1796,Image_1797.jpg
1797,Image_1798.jpg
1798,Image_1799.jpg


In [ ]:
learn.predict(img)

(Category tensor(1), tensor(1), tensor([1.1475e-08, 1.0000e+00, 2.0113e-07]))

In [ ]:
import tqdm,os,numpy

test['1']=np.nan
test['2']=np.nan
test['3']=np.nan
for count,rows in tqdm.tqdm(enumerate(test['filename'])):
  img=open_image(os.path.join('/content/food_classifier_data/test',test['filename'][count]))
  test['1'][count]=learn.predict(img)[-1].numpy()[0]
  test['2'][count]=learn.predict(img)[-1].numpy()[1]
  test['3'][count]=learn.predict(img)[-1].numpy()[2]
  #print(learn.predict(img))

1800it [02:56, 10.21it/s]


In [ ]:
test.to_csv("d121.csv",index=False)

In [ ]:
l2=learn.get_preds(DatasetType.Test)[0].numpy()

In [ ]:
import pickle
with open('r34.pb','wb') as f:
  pickle.dump(l1,f)

In [ ]:
test['prediction'].to_csv("submit_food.csv",index=False)

In [ ]:
data.classes

['Pizza', 'Softdrinks', 'burgers']

In [ ]:
test['prediction']=np.nan
for count,rows in tqdm.tqdm(enumerate(test['filename'])):
  test['1'][count]=data.classes[np.argmax(l[count])]

1800it [00:00, 13108.77it/s]


In [ ]:
test

,filename,prediction
0,Image_1.jpg,Softdrinks
1,Image_2.jpg,Softdrinks
2,Image_3.jpg,Pizza
3,Image_4.jpg,Pizza
4,Image_5.jpg,Pizza
...,...,...
1795,Image_1796.jpg,burgers
1796,Image_1797.jpg,burgers
1797,Image_1798.jpg,Pizza
1798,Image_1799.jpg,burgers


In [ ]:
test['prediction'].to_csv('tta.csv',index=False)